In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np

In [2]:
ds=mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
(x_train,y_train),(x_test,y_test)=ds
print(x_train.shape,x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [4]:
# Se hace al aplanado de las imágenes
x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

# Se modifica el tipo de datos a tipo flotante (para evitar conflictos)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Se normalizan los datos acorde a la intensidad de los píxeles
x_train /= 255.
x_test /= 255.

In [5]:
# Se realiza el one-hot enconding:

# Se definen el número de clases que hay en la base de datos
classes = 10

# Se realiza la conversión:
y_train = keras.utils.to_categorical(y_train,classes)
y_test = keras.utils.to_categorical(y_test,classes)

In [6]:
# Se configuran algunos aspectos del modelo:

# Número de neuronas de la capa oculta
n = 15

# Valor del Learning Rate
eta = 3.0
eta = float(eta) # Tiene que ser un dato de tipo flotante, pues de lo contrario
                 # tiene conflicto

# Número de épocas
epochs = 30

# Tamaño del mini-batch
mini_batch=10

In [7]:
# Se realiza la creación del modelo a como se había definido usando Numpy:
model = Sequential([
    Dense(n, activation='sigmoid', input_shape=(784,)),
    Dense(classes, activation='sigmoid')
])
# El modelo es secuencial, se van uniendo neurona por neurona de forma consecutiva

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 15)             │        11,775 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           165 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,260 (47.89 KB)

 Trainable params: 12,260 (47.89 KB)

 Non-trainable params: 0 (0.00 B)

Se creó un modelo con una capa oculta de 15 neuronas y una capa de salida de 10 neuronas, con las cuales se busca clasificar a las entradas (las imagenes aplanadas de los diez dígitos escritos a mano).

In [8]:
model.compile(loss='binary_crossentropy',optimizer=SGD(learning_rate=eta),
              metrics=['accuracy'])


In [9]:
history = model.fit(x_train, y_train,
                    batch_size = mini_batch,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test)
                    )

Epoch 1/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.5261 - loss: 0.2041 - val_accuracy: 0.8977 - val_loss: 0.0647
Epoch 2/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.9042 - loss: 0.0615 - val_accuracy: 0.9174 - val_loss: 0.0528
Epoch 3/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9192 - loss: 0.0510 - val_accuracy: 0.9252 - val_loss: 0.0470
Epoch 4/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9255 - loss: 0.0466 - val_accuracy: 0.9300 - val_loss: 0.0451
Epoch 5/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9338 - loss: 0.0431 - val_accuracy: 0.9265 - val_loss: 0.0468
Epoch 6/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9342 - loss: 0.0415 - val_accuracy: 0.9127 - val_loss: 0.0537
Epoch 7/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9364 - loss: 0.0393 - val_accuracy: 0.9313 - val_loss: 0.0425
Epoch 8/30
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9378 - loss: 0

In [10]:
mod = model.predict(x_test)

print("Resultado predicho: ")
print(np.where(mod[6] == np.max(mod[6])))
print("Resultado correcto: ")
print(np.where(y_test[6] == np.max(y_test[6])))


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Resultado predicho: 
(array([4]),)
Resultado correcto: 
(array([4]),)
